Run this code strictly in Colab.
Use the images given.
Install the neccessary libraries.

In [1]:
pip install numpy --upgrade

     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ---------------------------------------- 60.8/60.8 kB ? eta 0:00:00
   ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
   - -------------------------------------- 0.4/12.9 MB 13.5 MB/s eta 0:00:01
   -- ------------------------------------- 0.9/12.9 MB 14.0 MB/s eta 0:00:01
   ---- ----------------------------------- 1.3/12.9 MB 12.0 MB/s eta 0:00:01
   ----- ---------------------------------- 1.8/12.9 MB 12.3 MB/s eta 0:00:01
   ------ --------------------------------- 2.2/12.9 MB 12.7 MB/s eta 0:00:01
   -------- ------------------------------- 2.7/12.9 MB 12.2 MB/s eta 0:00:01
   --------- ------------------------------ 3.2/12.9 MB 12.1 MB/s eta 0:00:01
   ----------- ---------------------------- 3.6/12.9 MB 12.2 MB/s eta 0:00:01
   ------------ --------------------------- 4.1/12.9 MB 12.0 MB/s eta 0:00:01
   -------------- ------------------------- 4.6/12.9 MB 11.9 MB/s eta 0:00:01
   --

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ensorflow-intel 2.13.0 requires keras<2.14,>=2.13.1, but you have keras 2.15.0 which is incompatible.
ensorflow-intel 2.13.0 requires numpy<=1.24.3,>=1.22, but you have numpy 2.2.4 which is incompatible.
ensorflow-intel 2.13.0 requires tensorboard<2.14,>=2.13, but you have tensorboard 2.15.1 which is incompatible.
ensorflow-intel 2.13.0 requires tensorflow-estimator<2.14,>=2.13.0, but you have tensorflow-estimator 2.15.0 which is incompatible.
ensorflow-intel 2.13.0 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.12.2 which is incompatible.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.2.4 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.4 which is incompatible.
scipy 1.11.2 requires numpy<1.28.0,>=1.21.6, but you have numpy 2.2.4 whi

In [2]:
pip install mahotas

   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   -------- ------------------------------- 0.4/1.7 MB 23.7 MB/s eta 0:00:01
   --------------------- ------------------ 0.9/1.7 MB 11.9 MB/s eta 0:00:01
   -------------------------------- ------- 1.4/1.7 MB 12.9 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 11.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\saura\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip


In [4]:
# import the necessary packages
from scipy.spatial import distance as dist
import numpy as np
import mahotas
import cv2
from google.colab.patches import cv2_imshow as cvim
import imutils
  
def describe_shapes(image):
    # initialize the list of shape features
    shapeFeatures = []
  
    # convert the image to grayscale, blur it, and threshold it
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (13, 13), 0)
    thresh = cv2.threshold(blurred, 50, 255, cv2.THRESH_BINARY)[1]
  
    # perform a series of dilations and erosions to close holes
    # in the shapes
    thresh = cv2.dilate(thresh, None, iterations=4)
    thresh = cv2.erode(thresh, None, iterations=2)
  
    # detect contours in the edge map
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,
        cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
  
    # loop over the contours
    for c in cnts:
        # create an empty mask for the contour and draw it
        mask = np.zeros(image.shape[:2], dtype="uint8")
        cv2.drawContours(mask, [c], -1, 255, -1)
  
        # extract the bounding box ROI from the mask
        (x, y, w, h) = cv2.boundingRect(c)
        roi = mask[y:y + h, x:x + w]
  
        # compute Zernike Moments for the ROI and update the list
        # of shape features
        features = mahotas.features.zernike_moments(roi, cv2.minEnclosingCircle(c)[1], degree=8)
        shapeFeatures.append(features)
  
    # return a tuple of the contours and shapes
    return (cnts, shapeFeatures)
 
# load the reference image containing the object we want to detect,
# then describe the game region
refImage = cv2.imread("pokemon_red.png")
(_, gameFeatures) = describe_shapes(refImage)
  
# load the shapes image, then describe each of the images in the image
shapesImage = cv2.imread("shapes.png")
(cnts, shapeFeatures) = describe_shapes(shapesImage)
  
# compute the Euclidean distances between the video game features
# and all other shapes in the second image, then find index of the
# smallest distance
D = dist.cdist(gameFeatures, shapeFeatures)
i = np.argmin(D)
 
# loop over the contours in the shapes image
for (j, c) in enumerate(cnts):
    # if the index of the current contour does not equal the index
    # contour of the contour with the smallest distance, then draw
    # it on the output image
    if i != j:
        box = cv2.minAreaRect(c)
        box = np.int0(cv2.cv.BoxPoints(box) if imutils.is_cv2() else cv2.boxPoints(box))
        cv2.drawContours(shapesImage, [box], -1, (0, 0, 255), 2)
  
# draw the bounding box around the detected shape
box = cv2.minAreaRect(cnts[i])
box = np.int0(cv2.cv.BoxPoints(box) if imutils.is_cv2() else cv2.boxPoints(box))
cv2.drawContours(shapesImage, [box], -1, (0, 255, 0), 2)
(x, y, w, h) = cv2.boundingRect(cnts[i])
cv2.putText(shapesImage, "FOUND!", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9,
    (0, 255, 0), 3)
  
# show the output images
cvim(refImage)
cvim(shapesImage)
cv2.waitKey(0)

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject